In [ ]:
!pip install awswrangler

In [ ]:
import awswrangler as wr
import pandas as pd
from datetime import date

In [ ]:
path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_etl_inputs/dma_state_code/dma_state_code.csv'
output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/outputs/dma_state_code/'
df = pd.read_csv(path, header=0, dtype='str', delimiter='|')
df.rename(columns={'dma_state':'dma_name', 'dma_code':'dma_code'}, inplace=True)
df['etl_load_date'] = date.today().strftime('%Y-%m-%d')
try: 
    wr.s3.to_parquet(
            df=df,
            path=f'{output_path}{date.today()}/',
            dataset=True,
            database='sandbox_measurement_analyst',
            table='mars_dma_state_code_staging',
            mode="overwrite",
            index=False
        )
except Exception as e:
    raise e

In [ ]:
output_loc = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings_outputs/zip_dma_crossover/'
table_name = 'mars_zip_dma_crossover_staging'
path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/crossover_zip_code/Zip Universe.csv'
target_database = 'sandbox_measurement_analyst'
target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'


df = pd.read_csv(path, dtype='str', header=0)
df['County_Name'] = df['County_Name'].str.replace('\t', '')
df['TV_HH'] = df['TV_HH'].astype(float)

try:
    wr.s3.to_parquet(
        df=df,
        path=output_loc,
        dataset=True,
        database=target_database,
        table = table_name,
        mode="overwrite",
        index=False
    )
        
except Exception as e:
    print(f'error: {e}')
    raise e